In [27]:
import time
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from datetime import timedelta
import win32com.client as win32
import os
start_time = time.time()

In [28]:
# Declarar nombre archivos

reporte_cabecera = 'Cabecera 14-09.XLSX'
reporte_posiciones = 'Posiciones 14-09.XLSX'
reporte_tareas = 'Tareas 14-09.XLSX'
reporte_sto = 'Activacion 14-09.XLSX'
reporte_analisiscrc = 'Analisis KIT 2023-09-14.xlsx'
reporte_seguimiento = 'Reporte Seguimiento 14-09.XLSX'
reporte_afo = '2023-09-14 AFO AVANCE.xlsb'
reporte_dsm = 'Data 07.09.xlsx'
reporte_planificación = 'Planificación KIT.xlsb'

# Usar todas las OM que no han sido despachadas. Estatus KIT != Despachado

In [29]:
# Leer Archivos

afo = pd.read_excel(reporte_afo, engine='pyxlsb', sheet_name = 'Resumen x Ln', dtype = 'string', usecols= 'A:M')
analisis_crc = pd.read_excel(reporte_analisiscrc, sheet_name = 'PLAN', dtype = 'string')
cabecera = pd.read_excel(reporte_cabecera, dtype = 'string')
posiciones = pd.read_excel(reporte_posiciones , dtype = 'string')
tareas = pd.read_excel(reporte_tareas, dtype = 'string', usecols= 'A:P')
seguimiento = pd.read_excel(reporte_seguimiento, dtype = 'string')
dsm = pd.read_excel(reporte_dsm, dtype = 'string')
sto = pd.read_excel(reporte_sto, dtype = 'string', skipfooter = 3)
planif = pd.read_excel(reporte_planificación, engine='pyxlsb', sheet_name = 'Base_Planificación KIT', dtype = 'string')

In [30]:
# Filtro avanzado

# Filtramos en el AFO, todas las ordenes que tenemos en Planificación
analisis = afo.loc[(afo['Orden Superior'].isin(analisis_crc['Orden de Servicio']))]
analisis = analisis.reset_index()
analisis = analisis.drop(columns = 'index', axis = 1)
# Delta positivo
analisis['Delta'] = analisis['Delta'].str.replace("-","")

In [31]:
# 1. Arreglo archivo Tareas

# Eliminar filas sin documento
tareas = tareas.dropna(subset = 'Documento')
# Rellenar Status de tarea de almacén = Vacías con "Tarea Abierta"
tareas.loc[pd.isna(tareas['Status de tarea de almacén']), 'Status de tarea de almacén'] = 'Tarea Abierta'
# Rellenar Status de tarea de almacén = "C" con "Tarea Confirmada"
tareas.loc[tareas['Status de tarea de almacén'] == 'C', 'Status de tarea de almacén'] = 'Tarea Confirmada'
# Eliminar tareas Anuladas
tareas = tareas.loc[tareas['Status de tarea de almacén'] != 'A', :]
# Rellenar codigos de excepción
tareas.loc[pd.isna(tareas['Código de excepción']), 'Código de excepción'] = 'Sin Codigo de Excepción'
tareas.loc[tareas['Código de excepción'] == '****', 'Código de excepción']  = 'Sin Codigo de Excepción'
# ELiminar BIPI y BIFI
tareas = tareas.loc[tareas['Código de excepción'] != 'BIPI', :]
tareas = tareas.loc[tareas['Código de excepción'] != 'BIFI', :]
# Fecha de confirmación a Formato Datetime
tareas['Fecha confirmación'] = pd.to_datetime(tareas['Fecha confirmación'])
# Incluir solo las tareas del día y las tareas abiertas
tareas = tareas.loc[(tareas['Fecha confirmación'] == np.datetime64('today')) | 
                    (tareas['Status de tarea de almacén'] == 'Tarea Abierta')]
# Columna nueva para extrae la Hora de la Hora de Confirmación
tareas['Hora'] = tareas['Hora de confirmación'].str[:2]
# La llevamos a dtype Int
tareas['Hora'] = tareas['Hora'].astype(int)
# Filtrar Hora de confirmación >= 08.00 Hrs y Las tareas Abiertas
tareas = tareas.loc[(tareas['Hora'] >= 8) | (tareas['Status de tarea de almacén'] == 'Tarea Abierta')]
# Está acá queda lista la manipulación del archivo Tareas

In [32]:
# 2. Arreglo archivo Posiciones

# Seleccionar columnas de interes
posiciones_tareas = posiciones.loc[: , ['Documento','Orden de servicio','Operación de orden de servicio']]
# Operacion a formato "0000"
posiciones_tareas.loc[: , 'Operación de orden de servicio'] =\
list(map(lambda x: x.zfill(4), posiciones_tareas['Operación de orden de servicio']))
# Eliminar duplicados de Documento
posiciones_tareas = posiciones_tareas.drop_duplicates(subset = 'Documento', keep = 'first')

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13204\2712609874.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  posiciones_tareas.loc[: , 'Operación de orden de servicio'] =\


In [33]:
# 3. Cruzar posiciones y tareas. Con esto obtenemos la OS y la OP de los documentos PDO del archivo Tareas

posiciones_tareas = posiciones_tareas.set_index('Documento')
tareas = tareas.set_index('Documento')
tareas = tareas.join(posiciones_tareas, how = 'left')
tareas = tareas.reset_index()

In [34]:
# 4. Formato tabla dinamica

# 4.1 Pasar cantidades a formato numerico
tareas['Ctd.real dest.UMB'] = tareas['Ctd.real dest.UMB'].str.replace('0.0','0', regex = False)
tareas['Ctd.prev.proced.UMB'] = tareas['Ctd.prev.proced.UMB'].astype('int')
tareas['Ctd.real dest.UMB'] = tareas['Ctd.real dest.UMB'].astype('int')


# 4.2 Columna CTD Total
tareas['Ctd Total'] = np.where(tareas['Status de tarea de almacén'] == 'Tarea Confirmada',
                               tareas['Ctd.real dest.UMB'],tareas['Ctd.prev.proced.UMB'])

# 4.3 Columna OS+OP+MAT, para agregar Ctd Total, en funcion de OS+OP+MAT
tareas['OS+OP+MAT'] = tareas['Orden de servicio'] + tareas['Operación de orden de servicio'] + tareas['Producto']

# 4.4 Tabla Dinámica
td = tareas.pivot_table(values = ['Ctd Total'], index = ['OS+OP+MAT'], aggfunc=np.sum)
td = td.rename(columns = {'Ctd Total': 'Ctd Total_Td'})

# 4.5 Cruzar
tareas = tareas.set_index('OS+OP+MAT')
tareas = tareas.join(td, how = 'left')
tareas = tareas.reset_index()

# Todo el Dataframe a string
tareas = tareas.astype(str)

# OS+OP+MAT+CTD
tareas['OS+OP+MAT+CTD'] = tareas['Orden de servicio'] + tareas['Operación de orden de servicio'] +\
tareas['Producto'] + tareas['Ctd Total_Td']

In [35]:
analisis

,Orden Superior,Orden,Operación,Material,TSolicitado,TConsumido,TFaltanteBO,TRecibido,OH 8034,OH Otros,OH Total,Delta,ST Linea
0,000001016271,000001016271,0010,SK1-MF988H-UP.007,0,0,0,0,<NA>,<NA>,0,0,Completa
1,000001016271,000001016271,0020,1378837.000,0,0,0,0,<NA>,<NA>,0,0,Completa
2,000001016271,000001016271,0020,1434512.000,1,0,1,1,<NA>,<NA>,0,1,Pendiente Total
3,000001016271,000001016271,0020,1481546.000,0,0,0,0,<NA>,<NA>,0,0,Completa
4,000001016271,000001016271,0020,1752853.000,1,0,1,1,<NA>,<NA>,0,1,Pendiente Total
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11206,000001022513,000001022513,0020,3787609.000,1,0,1,0,<NA>,<NA>,0,1,Pendiente Total
11207,000001022513,000001022513,0020,4238524.000,1,0,1,0,<NA>,<NA>,0,1,Pendiente Total
11208,000001022513,000001022513,0020,5746249.000,1,0,1,0,<NA>,<NA>,0,1,Pendiente Total
11209,000001022513,000001022513,0020,5F8000.000,1,0,1,0,<NA>,<NA>,0,1,Pendiente Total


In [36]:
# 5. Cruzar con AFO (Planilla Análisis)

# 5.1 Cruce con Tareas Confirmadas
analisis['OS+OP+MAT+CTD'] = analisis['Orden'] + analisis['Operación'] + analisis['Material'] + analisis['Delta']
tareas = tareas[['OS+OP+MAT+CTD','Status de tarea de almacén','Tipo de stocks','UMp destino','Ubic.procedencia',
                 'Fecha confirmación','Hora de confirmación','Documento','Ctd.prev.proced.UMB', 'Ctd.real dest.UMB',
                 'Orden de almacén']]

# Key Column OS+OP+MAT+CTD
analisis = analisis.set_index('OS+OP+MAT+CTD')
tareas = tareas.set_index('OS+OP+MAT+CTD')

# Cruce
analisis = analisis.join(tareas, how = 'left')
# Rellenar valores NA en Columnas UMp destino y Fecha confirmación
analisis.loc[analisis['UMp destino'] == '<NA>', 'UMp destino'] = ''
analisis.loc[analisis['Fecha confirmación'] == 'NaT', 'Fecha confirmación'] = ''

In [37]:
# 6. Cruce AFO y lineas con Entrega de Salida
posiciones_analisis = posiciones.loc[: , ['Documento','Documento original de ERP','Pedido de compras','Posición de pedido',
                                 'Orden de servicio','Operación de orden de servicio','Prod.','Ctd.','Ubicación',
                                 'Tipo de stock','Status de picking']]
# Operacion a formato "0000"
posiciones_analisis.loc[: , 'Operación de orden de servicio'] =\
list(map(lambda x: x.zfill(4), posiciones_analisis['Operación de orden de servicio']))

# OS+OP+MAT+CTD
posiciones_analisis['OS+OP+MAT'] = posiciones_analisis['Orden de servicio'] +\
posiciones_analisis['Operación de orden de servicio'] + posiciones_analisis['Prod.']

# Solo considerar Status Picking = No Iniciado
posiciones_analisis = posiciones_analisis.loc[posiciones_analisis['Status de picking'] == 'No iniciado', :]

# Tabla dinamica para las cantidades agregadas

posiciones_analisis['Ctd.'] = posiciones_analisis['Ctd.'].astype('float')
posiciones_analisis['Ctd.'] = posiciones_analisis['Ctd.'].astype('int') # Columna Cantidad a int
td2 = posiciones_analisis.pivot_table(values = ['Ctd.'], index = ['OS+OP+MAT'], aggfunc=np.sum)
td2 = td2.rename({'Ctd.': 'Ctd Total'}, axis = 1)

# Join Tabla Dinamica
posiciones_analisis = posiciones_analisis.set_index('OS+OP+MAT')
posiciones_analisis = posiciones_analisis.join(td2, how = 'left')
posiciones_analisis = posiciones_analisis.reset_index()
posiciones_analisis = posiciones_analisis.drop('OS+OP+MAT', axis = 1)

# OS+OP+MAT+CTD, usando Ctd Total
posiciones_analisis = posiciones_analisis.astype('string')
posiciones_analisis['OS+OP+MAT+CTD'] = posiciones_analisis['Orden de servicio'] +\
posiciones_analisis['Operación de orden de servicio'] + posiciones_analisis['Prod.'] + posiciones_analisis['Ctd Total']

# Eliminar columnas que no nos interesan para el cruce
posiciones_analisis = posiciones_analisis.drop(labels = ['Orden de servicio','Operación de orden de servicio',
                                                         'Prod.','Ctd Total'], axis = 1)

# Renombrar columnas para cruce
posiciones_analisis = posiciones_analisis.rename(columns={"Documento": "Documento_Posiciones",
                                                          "Ubicación": "Ubicación_Posiciones",
                                                         "Tipo de stock": "Tipo de stock_Posiciones"})
# Join AFO y lineas con Entrega de Salida

analisis = analisis.reset_index()

# Lineas Completas y Completas SobreConsumo. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Completa') | (analisis['ST Linea'] == 'Completa - Sobre Consumo'), 'OS+OP+MAT+CTD'] = ''

# Lineas Pendientes Parcial con Status Tarea de Almacen Vacía. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Pendiente Parcial') &\
             (pd.isna(analisis['Status de tarea de almacén']) == False),'OS+OP+MAT+CTD'] = ''

# Lineas Pendiente Total con Status Tarea de Almacen Vacía. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Pendiente Total') &\
             (pd.isna(analisis['Status de tarea de almacén']) == False), 'OS+OP+MAT+CTD'] = ''

# Join
posiciones_analisis = posiciones_analisis.set_index('OS+OP+MAT+CTD')
analisis = analisis.set_index('OS+OP+MAT+CTD')
analisis = analisis.join(posiciones_analisis, how = 'left')

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13204\3294363258.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  posiciones_analisis.loc[: , 'Operación de orden de servicio'] =\


In [38]:
# 7. Juntar Documentos , Ubicaciones, Tipo de Stock y Cantidades

# Documentos
analisis.loc[pd.isna(analisis['Documento']), 'Documento'] = ''
analisis.loc[pd.isna(analisis['Documento_Posiciones']), 'Documento_Posiciones'] = ''
analisis['Documento PDO'] = analisis['Documento'] + analisis['Documento_Posiciones']

# Ubicaciones
analisis.loc[pd.isna(analisis['Ubic.procedencia']), 'Ubic.procedencia'] = ''
analisis.loc[pd.isna(analisis['Ubicación_Posiciones']), 'Ubicación_Posiciones'] = ''
analisis['Ubicación'] = analisis['Ubic.procedencia'] + analisis['Ubicación_Posiciones']

# Tipo de Stock
analisis.loc[pd.isna(analisis['Tipo de stocks']), 'Tipo de stocks'] = ''
analisis.loc[pd.isna(analisis['Tipo de stock_Posiciones']), 'Tipo de stock_Posiciones'] = ''
analisis['Tipo de Stock'] = analisis['Tipo de stocks'] + analisis['Tipo de stock_Posiciones']

# Cantidades
analisis = analisis.astype('string')
analisis = analisis.fillna('')
analisis.loc[analisis['Status de tarea de almacén'] == 'Tarea Abierta', 'Ctd.real dest.UMB'] = ''
analisis.loc[analisis['Status de tarea de almacén'] == 'Tarea Confirmada', 'Ctd.prev.proced.UMB'] = ''
analisis['Cantidad'] = analisis['Ctd.prev.proced.UMB'] + analisis['Ctd.real dest.UMB'] + analisis['Ctd.']

# Status de picking = No Iniciado. Le agregaremos "Picking"
analisis.loc[analisis['Status de picking'] == 'No iniciado', 'Status de tarea de almacén'] = 'Picking No iniciado'

# ELiminar columnas inncesarias
analisis = analisis.drop(labels = ['Documento', 'Documento_Posiciones', 'Ubic.procedencia', 'Ubicación_Posiciones',
                                  'Tipo de stocks', 'Tipo de stock_Posiciones','Status de picking','Ctd.prev.proced.UMB',
                                  'Ctd.real dest.UMB', 'Ctd.'], axis = 1)

# Reset index y eliminar OS+OP+MAT+CTD
analisis = analisis.reset_index()
analisis = analisis.drop(columns = 'OS+OP+MAT+CTD', axis = 1)

In [39]:
# 8. Cruce con reporte seguimiento

# DF Base

# Arreglar formato Cantidad entrega entrando
seguimiento['Cantidad'] = seguimiento['Cantidad'].astype('float')
seguimiento['Cantidad'] = seguimiento['Cantidad'].astype('int64')
seguimiento['Cantidad'] = seguimiento['Cantidad'].astype('string')

# OS+OP+MAT+CTD
seguimiento['Orden'] = seguimiento['Orden'].apply(lambda x: x.zfill(12))
seguimiento['OS+OP+MAT+CTD'] = seguimiento['Orden'] + seguimiento['Operación'] +\
                               seguimiento['Material'] + seguimiento['Cantidad']

# DF Filtrado
seguimiento = seguimiento.loc[: ,['OS+OP+MAT+CTD', 'Documento compras', 'Posición', 'Entrega','Cantidad',
                                  'Posición.1','Unidad manipulación', 'Proveedor', 'Recibido']]

# Renombrar columnas
seguimiento = seguimiento.rename({'Posición':'POS_PO','Posición.1':'POS_E.E','Unidad manipulación':'Case'}, axis = 1)

# Solo las lineas aún no recibidas
seguimiento = seguimiento.loc[seguimiento['Recibido'] == '0', :]

# ELiminar Centro 5055 y 5056
seguimiento = seguimiento.loc[seguimiento['Proveedor'] != '5055']
seguimiento = seguimiento.loc[seguimiento['Proveedor'] != '5056']

# OS+OP+MAT+CTD para analisis
analisis['OS+OP+MAT+CTD'] = analisis['Orden'] + analisis['Operación'] + analisis['Material'] + analisis['Delta']

# Lineas Completas y Completas SobreConsumo. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Completa') | (analisis['ST Linea'] == 'Completa - Sobre Consumo'), 'OS+OP+MAT+CTD'] = ''

# Lineas Pendientes Parcial con Status Tarea de Almacen Vacía. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Pendiente Parcial') &\
             (analisis['Status de tarea de almacén'] != ''),'OS+OP+MAT+CTD'] = ''

# Lineas Pendiente Total con Status Tarea de Almacen Vacía. OS+OP+MAT+CTD = 0
analisis.loc[(analisis['ST Linea'] == 'Pendiente Total') &\
             (analisis['Status de tarea de almacén'] != ''), 'OS+OP+MAT+CTD'] = ''

# Eliminar duplicados OS+OP+MAT+CTD seguimienti
seguimiento = seguimiento.drop_duplicates(subset = 'OS+OP+MAT+CTD', keep = False)

# Key Columns
analisis = analisis.set_index('OS+OP+MAT+CTD')
seguimiento = seguimiento.set_index('OS+OP+MAT+CTD')

# Join
analisis = analisis.join(seguimiento, how = 'left', rsuffix = '_Seguimiento')
analisis = analisis.reset_index()
analisis = analisis.drop(labels = 'Recibido', axis = 1)

# ELiminar OS+OP+MAT+CTD
analisis = analisis.drop('OS+OP+MAT+CTD', axis = 1)

# ST Línea Pendiente = Fábrica, si Dcto Compras empieza con 8*

analisis.loc[(analisis['ST Linea'].str.startswith('Pendiente')) & (analisis['Documento compras'].str.startswith('8')),
            'Status de tarea de almacén'] = 'Fábrica'

analisis.loc[(analisis['ST Linea'].str.startswith('Pendiente')) & (analisis['Documento compras'].str.startswith('4')),
            'Status de tarea de almacén'] = 'Nacional'

In [40]:
# 9. Contenedores

dsm_contenedores = dsm.loc[(dsm['Orden de Compra'].str.startswith('4')) & (dsm['Centro Suministrador'] == '5055') &\
                    (dsm['Estado Final'] == 'Atendido nacional sin entrega de salida generada'),:]

dsm_contenedores = dsm_contenedores.loc[:,['Orden de Servicio', 'Operación', 'Orden de Compra', 'OC Posición', 'Material',
                                    'Cantidad necesaria']]

dsm_contenedores.loc[: , 'Orden de Servicio'] = list(map(lambda x: x.zfill(12), dsm_contenedores['Orden de Servicio']))
dsm_contenedores.loc[: , 'Operación'] = list(map(lambda x: x.zfill(4), dsm_contenedores['Operación']))

dsm_contenedores['OS+OP+MAT+CTD'] = dsm_contenedores['Orden de Servicio'] + dsm_contenedores['Operación'] +\
                                    dsm_contenedores['Material'] + dsm_contenedores['Cantidad necesaria']

# Eliminar Duplicados
dsm_contenedores = dsm_contenedores.drop_duplicates('OS+OP+MAT+CTD', keep = 'first')

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13204\463562997.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dsm_contenedores.loc[: , 'Orden de Servicio'] = list(map(lambda x: x.zfill(12), dsm_contenedores['Orden de Servicio']))
C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13204\463562997.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dsm_contenedores.loc[: , 'Operación'] = list(map(lambda x: x.zfill(4), dsm_contenedores['Operación']))


In [41]:

# Ajustes Análisis para cruzar con contenedores
analisis['OS+OP+MAT+CTD'] = ''
analisis['OS+OP+MAT+CTD'] = np.where(analisis['Status de tarea de almacén'] == '',
                            analisis['Orden']+analisis['Operación']+analisis['Material']+analisis['Delta'],'')
# Eliminar OS+OP+MAT+CTD para las líneas Completas
analisis.loc[((analisis['ST Linea'] == 'Completa') | (analisis['ST Linea'] == 'Completa - Sobre Consumo')),
             'OS+OP+MAT+CTD'] = ''

# Cruce Contenedores y Analisis
analisis = analisis.set_index('OS+OP+MAT+CTD')
dsm_contenedores = dsm_contenedores.set_index('OS+OP+MAT+CTD')
dsm_contenedores = dsm_contenedores.drop(['Orden de Servicio', 'Operación', 'Material', 'Cantidad necesaria'], axis = 1)

# Crear columna Contenedor antes de cruzar
dsm_contenedores['Contenedor'] = 'Contenedor'

# Join
analisis = analisis.join(dsm_contenedores, how = 'left')
analisis = analisis.reset_index()

# Rellenar ST Línea Pendiente = Contenedor, para los Contenedores Identificados
analisis.loc[analisis['Contenedor'] == 'Contenedor', 'Status de tarea de almacén'] = 'Contenedor'

In [42]:
# 10. Ajustes Finales

# Unir Documentos de Compra
analisis = analisis.fillna('')
analisis['Dcto Compras'] = analisis['Pedido de compras'] + analisis['Documento compras'] + analisis['Orden de Compra']
analisis['Posición_Dcto_Compras'] = analisis['Posición de pedido'] + analisis['POS_PO'] + analisis['OC Posición']

# Unir cantidad Reporte Seguimiento (lineas Fábrica y Nacionales)
analisis['Cantidad Total'] = analisis['Cantidad'] + analisis['Cantidad_Seguimiento']

# Eliminar Columnas Auxiliares
analisis = analisis.drop(['OS+OP+MAT+CTD', 'Documento compras', 'POS_PO', 'Orden de Compra', 'OC Posición',
                         'Pedido de compras', 'Posición de pedido','Cantidad', 'Cantidad_Seguimiento'], axis = 1)

# Renombrar Columnas
analisis = analisis.rename(columns = {'Dcto Compras':'Documento de Compras',
                                      'Posición_Dcto_Compras':'Posición_PO',
                                      'Cantidad Total': 'Cantidad'})

In [43]:
# Activaciones. Revisar Lineas Completas, con TSol = 0

# OS+OP+MAT para las Lineas Completas, con TSol = 0
analisis['OS+OP+MAT'] = np.where((analisis['ST Linea'] == 'Completa') & (analisis['TSolicitado'] == '0'),
                                analisis['Orden'] + analisis['Operación'] + analisis['Material'], '')

# Sto OS+OP+MAT y columna para cruce
sto['Orden'] = sto['Orden'].apply(lambda x: x.zfill(12))
sto['OS+OP+MAT'] = sto['Orden'] + sto['Op.'] + sto['Material']
sto['Activacion'] = 'Activada'
sto = sto.loc[: , ['OS+OP+MAT','Activacion']]

# Cruce
sto = sto.set_index('OS+OP+MAT')
analisis = analisis.set_index('OS+OP+MAT')
analisis = analisis.join(sto, how = 'left')
analisis = analisis.reset_index()
analisis = analisis.drop('OS+OP+MAT', axis = 1)

# Rellenar Status de tarea de almacén
analisis.loc[analisis['Activacion'] == 'Activada', 'Status de tarea de almacén'] = 'Revisar Activación'
analisis = analisis.drop('Activacion', axis = 1)

In [44]:
# Cruzar con planilla planificación

#Elegir columnas de interes para cruce
planif_cruce = planif.loc[:,['OV/Pedido','Pos','Cliente','Fecha Requerimiento','NPKit','Grupo de Sub Orden','OM',
                             'SubKIT','Week Despacho','Fecha Despacho\nPlanificada','Fecha Max Picking\nPlanificada',
                             'Fecha Armado \nPlanificada']]

# Set Index
planif_cruce = planif_cruce.set_index('OM')
analisis = analisis.set_index('Orden')

# Cruce
analisis = analisis.join(planif_cruce, how = 'left')

In [45]:
# 36 Descargar Archivo
today = date.today()
fecha_hoy = today.strftime("%Y-%m-%d")
Nombre_Archivo = fecha_hoy + ' ANALISIS KIT.xlsx'
with pd.ExcelWriter(Nombre_Archivo) as writer:  
    analisis.to_excel(writer, sheet_name='AFO', index = False)

In [46]:
# 37 Abrir excel y modificarlo

# Abrir Instancia Excel
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True

# Abrir On Hand Completo (Libro)
path = os.path.abspath(Nombre_Archivo)
wb = xlApp.Workbooks.Open(path)

# Ajustar Columnas
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns.AutoFit()

# Guardar como XLSB
wb.SaveAs(Filename = os.path.dirname(path) + '\\' + fecha_hoy + ' ANALISIS KIT', FileFormat = 50)